# configuration file testing and setup

In [5]:
import new_cfg
import ConfigParser
config = ConfigParser.ConfigParser()

In [15]:
reload(new_cfg)
rc_fh = new_cfg.new_rc()

In [16]:
config.read(rc_fh)
for sec in config.sections():
    print '[' + sec + ']'
    for item in config.items(sec):
        print item[0], '=', item[1]
    print ''

[Data]
data_cut = 1
halo = halo02
d_mpc = 4.0
filter_type = h158

[PATH]
curdir_path = C:\Users\swc21\GitHub\skysearcher\skysearcher
curdir_name = skysearcher
pardir_path = C:\Users\swc21\GitHub\skysearcher
pardir_name = skysearcher
data_dir = C:\Users\swc21\GitHub\skysearcher\data
plot_dir = C:\Users\swc21\GitHub\skysearcher\data\plots
table_dir = C:\Users\swc21\GitHub\skysearcher\data\tables
grid_dir = C:\Users\swc21\GitHub\skysearcher\data\grids
fh_prefix = halo02_4.0_h158
grid_file_designator = grid
grid_ext = .npy
table_file_designator = table
table_format = hdf5
table_ext = .hdf5
table_hdf5_path = data

[Program Units]
units_radius = kpc
units_annulus = degree

[Search Extent]
r_start = 10
r_stop = 275
annulus_scale = 0.1
annulus_phi_step = 1
xbox_min = 10.0

[Accept Reject]
min_boxes = 25

[Plots]
full_projection = True
heatmap = True
regions_together = True



# skysearch_lib testing

In [3]:
import skysearch_lib as ss_lib
import ConfigParser
#import ConfigParser

In [13]:
config = ConfigParser.ConfigParser()
config.read(ss_lib.sortout_rcfile())
for sec in config.sections():
    print '[' + sec + ']'
    for item in config.items(sec):
        print item[0], '=', item[1]
    print ''

[Data]
data_cut = 1
halo = halo02
d_mpc = 4.0
filter_type = h158

[PATH]
curdir_path = C:\Users\swc21\GitHub\skysearcher\skysearcher
curdir_name = skysearcher
pardir_path = C:\Users\swc21\GitHub\skysearcher
pardir_name = skysearcher
data_dir = C:\Users\swc21\GitHub\skysearcher\data
plot_dir = C:\Users\swc21\GitHub\skysearcher\data\plots
table_dir = C:\Users\swc21\GitHub\skysearcher\data\tables
grid_dir = C:\Users\swc21\GitHub\skysearcher\data\grids
fh_prefix = halo02_4.0_h158
grid_file_designator = grid
grid_ext = .npy
table_file_designator = table
table_format = hdf5
table_ext = .hdf5

[Program Units]
units_radius = kpc
units_annulus = degree

[Search Extent]
r_start = 10
r_stop = 275
annulus_scale = 0.1
annulus_phi_step = 1
xbox_min = 10.0

[Accept Reject]
min_boxes = 25

[Plots]
full_projection = True
heatmap = True
regions_together = True



# file system support testing

In [ ]:
import os

In [ ]:
# string path to this (current) directory
curdir_path = os.path.abspath(os.path.curdir)

In [ ]:
# parent directory path and current directory name
pardir_path, curdir_name = os.path.split(curdir_path)

In [ ]:
# parent directory name
pardir_name = os.path.split(pardir_path)[1]

In [ ]:
# list of files in this directory with .cfg ext
[f for f in os.listdir(os.path.abspath(os.path.curdir)) if os.path.splitext(f)[1] == '.cfg']

In [ ]:
dmpc = '4.0'
filter_type = 'h158'
halo ='halo02'
file_type = 'grid'
ext = 'npy'

In [ ]:
'_'.join([halo, dmpc, filter_type, file_type]) + os.path.extsep + ext

In [ ]:
'_'.join([halo, dmpc, filter_type])#, file_type])

# annuluar and radial segment limits

In [31]:
import os
import numpy as np
import skysearch_lib as ss_lib
from astropy.table import Table

In [19]:
# The starting degree of this radius' annulus (phi_start). 
phi_start = 0.0

# The ending degree (phi_stop).
phi_stop = 360.0

# <<ss_lib.annulus_phi_step>> - For units of degrees, this is the number 
# we dived 360.0 by to get the evaluation region's segment size.  The 
# larger this number, the smaller the segment.  How do smaller or larger 
# segment sizes effect this algorithm? (float)

# The size of the annular segment to evaluate (phi_step). (float)
phi_step = 360.0 // ss_lib.annulus_phi_step

In [20]:
print phi_step

360.0


In [27]:
arr, step = np.linspace(start=phi_start, stop=phi_stop, num=phi_step, endpoint=True, retstep=True, dtype=np.float16)


# custom generator function

In [115]:
from matplotlib import pyplot as plt
import numpy as np

In [141]:
def regions(r_0, r_1, r_step, r_scale, deg_0, deg_1, deg_step):
    
    for radius in xrange(r_0, r_1, r_step):
        
        region = radius * r_scale
        r_in = radius - region
        r_out = radius + region
        #r_in = radius 
        #r_out = radius + r_step
        
        for annular_segment in np.linspace(deg_0, deg_1, deg_step, dtype=np.float16):
              
            yield r_in, r_out, annular_segment

In [37]:
table_dir = 'C:/Users/swc21/skysurvey/tables/merged_tables/'
#print os.listdir(table_dir)
table = Table.read(table_dir + 'halo08.hdf5', format='hdf5', path='data')

In [94]:
_regions = regions(10, 275, 10, 0.1, 0, 360, 15)

In [54]:
_regions.next()

(45.0, 55.0, -181.0, -179.0)

In [147]:
r_0 = 10
r_1 = 275
r_step = 1
r_scale = 0.1
deg_0 = 0.0
deg_1 = 360.0
deg_step = 180.0
patch = 360.0 / deg_step
wdth = (patch * deg_mod) * 1.065

In [148]:
fig = plt.figure(figsize=(10,10))
#ax = fig.add_subplot(111)
rect = [0.1, 0.1, 0.8, 0.8]
deg_mod = 2.0 * (np.pi / 360.0)
plr = fig.add_axes(rect, polar=True, alpha=.2, facecolor=None, frame_on=False)


for r_in, r_out, annular_segment in regions(r_0, r_1, r_step, r_scale, deg_0, deg_1, deg_step):
    lft = annular_segment * deg_mod
    
    plr.bar(
        left=lft + wdth, 
        height=r_out - r_in, 
        width=wdth, 
        bottom=r_in,
        alpha=.2)
    
plr.set_thetagrids(range(0,360,15))
plr.set_ylim([0, 300])



ax = fig.add_axes(rect, alpha=.2, facecolor=None, frame_on=True)
ax.set_ylim([0, 600])
ax.set_xlim([0, 600])
ax.axis('off')
ax.set_aspect('equal')

plt.show()

#for r_in, r_out, deg_start, deg_stop in _regions:
    

KeyboardInterrupt: 

0.017453292519943295